In [1]:
import numpy as np
import torch
import pandas as pd

import matplotlib.pyplot as plt

from collections import namedtuple
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import data_processing as dp
import discriminator_torch as di
import discriminator_utilities as du
import evaluation as ev
import vocab as vo

In [425]:
vocab1, _, _, _ = dp.load_problem(lang='ru')
vocab2, _, _, _ = dp.load_problem(lang='en')

In [426]:
len(vocab1.embeddings), len(vocab2.embeddings)

(176032, 47404)

In [427]:
!python -c "import torch; print(torch.cuda.is_available(), torch.cuda.device_count())"

True 1


In [428]:
!pwd

/media/itasarom/7158f0d7-af1d-45d4-855f-86204aed5a8b/Projects/diplom/simple_classifier


In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.device_count()

1

In [431]:
import os
os.environ["LD_LIBRARY_PATH"]

'/home/itasarom/Documents/Projects/link_to_projects/apertium/apertium_root//lib:/usr/local/cuda-9.0/lib64::/usr/lib/x86_64-linux-gnu:/usr/local/lib:/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu:/usr/lib/nvidia-387'

In [432]:
import sys
sys.executable

'/home/itasarom/.programs/anaconda2/envs/torch/bin/python'

In [433]:
sys.argv[0]

'/home/itasarom/.programs/anaconda2/envs/torch/lib/python3.5/site-packages/ipykernel/__main__.py'

In [434]:
del cls

In [435]:
import gc
gc.collect()

0

In [436]:
batch_sampler = du.BatchSamplerStupid(vocab1, vocab2)

In [437]:
cls = di.Discriminator(vocab.embedding_dim)

In [438]:
cls = cls.cuda()

In [439]:
# del cls
torch.cuda.empty_cache()

In [440]:
# torch.cuda.

In [441]:
cls

Discriminator(
  (actual_model): Sequential(
    (0): Linear(in_features=100, out_features=2048)
    (1): Linear(in_features=2048, out_features=512)
    (2): Linear(in_features=512, out_features=2)
  )
  (softmax): Softmax()
  (loss_function): CrossEntropyLoss(
  )
)

In [442]:
trainer = du.Trainer(cls)

In [443]:
trainer.train(batch_sampler=batch_sampler, n_epochs=10, max_iterations=10**5, save_path="./checkpoints/checkpoint.ckpt", save_every=10)

9it [00:00, 87.37it/s]

Starting epoch  0


6919it [01:17, 89.57it/s]
10it [00:00, 91.59it/s]

After epoch 0 validation_loss = 0.0, train_loss = 0.0016552844317629933
Starting epoch  1


6919it [01:16, 90.19it/s]
10it [00:00, 94.06it/s]

...

After epoch 7 validation_loss = 0.0, train_loss = 0.0005764097440987825
Starting epoch  8


6919it [01:25, 80.62it/s]
9it [00:00, 85.87it/s]

After epoch 8 validation_loss = 0.0, train_loss = 0.0003188792907167226
Starting epoch  9


6919it [01:25, 81.12it/s]


After epoch 9 validation_loss = 0.0, train_loss = 0.00045385671546682715


In [444]:
trainer.train_loss

[0.0016552844,
 9.00829e-05,
 0.000536504,
 0.0006403176,
 0.002178609,
 0.0018994363,
 0.0013635004,
 0.00057640974,
 0.0003188793,
 0.00045385672]

In [445]:
x,  y = batch_sampler.get_test()

In [446]:
logits, probs = cls.forward(x)

In [447]:
pred = probs.cpu().data.numpy().argmax(axis=1)

In [448]:
np.count_nonzero(pred == y.ravel())/len(y)

1.0

In [449]:
cls.get_loss(x, y)

Variable containing:
 0
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [450]:
np.count_nonzero(y)/len(y)

0.7890625

In [451]:
len(vocab1.embeddings)

176032

In [452]:
len(vocab2.embeddings)

47404

In [453]:
import sklearn.metrics as skm

In [454]:
skm.accuracy_score(pred, y)

1.0

In [455]:
skm.f1_score(pred, y)

1.0

In [456]:
np.any(pred != 0)

True